In [ ]:
%pip install yolov5

In [ ]:
import torch
import yaml
from yolov5.utils.general import increment_path, non_max_suppression
from yolov5.utils.loss import ComputeLoss
from yolov5.models.yolo import Model
from yolov5.utils.torch_utils import select_device
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from yolov5.utils.datasets import LoadImagesAndLabels

# Parameters
INPUT_SIZE = 250  # Update to match your image size
NUM_CLASSES = 3  # Update this to match your dataset
BATCH_SIZE = 32
EPOCHS = 60
LEARNING_RATE = 0.0001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define paths
train_data_path = '../yolo_data_v2/train/images'
val_data_path = '../yolo_data_v2/val/images'
weights_path = 'yolov5n.pt'  # Using YOLOv5 nano model

# Create data.yaml for YOLO
data_config = {
    'train': train_data_path,
    'val': val_data_path,
    'nc': NUM_CLASSES,
    'names': ['class0', 'class1', 'class2']  # Update class names as needed
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_config, f)

# Load the YOLOv5 model
model = Model(cfg='yolov5/models/yolov5n.yaml', nc=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(weights_path, map_location=DEVICE)['model'])

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = ComputeLoss(model)

# Load datasets
train_dataset = LoadImagesAndLabels(data_config['train'], img_size=INPUT_SIZE, batch_size=BATCH_SIZE, augment=True, rect=False)
val_dataset = LoadImagesAndLabels(data_config['val'], img_size=INPUT_SIZE, batch_size=BATCH_SIZE, augment=False, rect=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=val_dataset.collate_fn)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for imgs, targets, paths, _ in train_loader:
        imgs = imgs.to(DEVICE).float() / 255.0
        targets = targets.to(DEVICE)

        optimizer.zero_grad()
        pred = model(imgs)
        loss, _ = criterion(pred, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss/len(train_loader)}")

# Validation loop
model.eval()
val_loss = 0
all_true_labels = []
all_pred_labels = []

with torch.no_grad():
    for imgs, targets, paths, shapes in val_loader:
        imgs = imgs.to(DEVICE).float() / 255.0
        targets = targets.to(DEVICE)

        pred = model(imgs)
        loss, _ = criterion(pred, targets)
        val_loss += loss.item()

        # Collect predictions and true labels for evaluation
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
        for p, t in zip(pred, targets):
            if p is not None:
                all_pred_labels.extend(p[:, -1].cpu().numpy())
            all_true_labels.extend(t[:, -1].cpu().numpy())

# Calculate mAP, precision, recall, F1 score
# The below is a placeholder for actual mAP calculation
precision, recall, f1 = 0.0, 0.0, 0.0  # Update with actual computation
print(f"Validation Loss: {val_loss/len(val_loader)}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Visualization of predictions
def visualize_predictions(images, predictions):
    for img, pred in zip(images, predictions):
        img = img.permute(1, 2, 0).cpu().numpy()
        plt.imshow(img)
        if pred is not None:
            for box in pred:
                x1, y1, x2, y2, conf, cls = box
                plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2))
                plt.text(x1, y1 - 2, f'{int(cls)} {conf:.2f}', bbox=dict(facecolor='red', alpha=0.5), fontsize=12, color='white')
        plt.show()

# Run inference on validation set to visualize predictions
model.eval()
with torch.no_grad():
    for imgs, targets, paths, shapes in val_loader:
        imgs = imgs.to(DEVICE).float() / 255.0
        pred = model(imgs)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
        visualize_predictions(imgs, pred)
        break  # Visualize only the first batch
